In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5):
        self.data = []
        self.sequence_length = sequence_length
        self.max_speed = 0  # Variable to store the maximum speed found
        self.preprocess_data(directory)
        # Calculate number of classes based on the range from 30 to 105
        self.num_classes = (105 - 30) // 10 + 1  # From class 30-39, ..., 100-105

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                # Update maximum speed if current speed is greater
                if speed > self.max_speed:
                    self.max_speed = speed
                # Convert speed to class, starting range from 30
                if speed < 30:
                    speed_class = 0  # Handle speeds below 30 as class 0
                else:
                    speed_class = (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(',')[:6])
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1,y1,x2,y2,(x1+x2)/2, (y1+y2)/2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = []
                            for i in range(1, len(frames)):
                                current_frame = frames[i]
                                previous_frame = frames[i-1]
                                data_d = []
                                for i in range(4):
                                    data_d.append(current_frame[i])
                                for j in range(4,6):
                                    data_d.append(current_frame[j] - previous_frame[j])
                                features.append(data_d)
                                
                            for i in range(len(features) - self.sequence_length + 1):
                                sequence = features[i:i + self.sequence_length]
                                self.data.append((sequence, speed_class))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)


In [2]:
train_dataset = SpeedDataset('outputs', sequence_length=5)
test_dataset = SpeedDataset('test', sequence_length=5)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [3]:
train_dataset[2]

(tensor([[ 6.0654e+02,  4.6902e+02,  6.6354e+02,  5.1220e+02,  1.3805e+00,
           3.0359e-01],
         [ 6.0735e+02,  4.7290e+02,  6.6468e+02,  5.1298e+02,  9.7501e-01,
           2.3286e+00],
         [ 6.0807e+02,  4.7742e+02,  6.6695e+02,  5.1390e+02,  1.4925e+00,
           2.7197e+00],
         [ 6.1008e+02,  4.7632e+02,  6.6941e+02,  5.1447e+02,  2.2345e+00,
          -2.6654e-01],
         [ 6.1158e+02,  4.7196e+02,  6.7179e+02,  5.1450e+02,  1.9440e+00,
          -2.1621e+00]]),
 tensor(7))